In [23]:
#!pip install Keras

In [24]:
from keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
from keras.applications.inception_v3 import preprocess_input as preprocess_input_inception
from keras.applications.imagenet_utils import decode_predictions
import numpy as np
import os
import json

# Load VGG16 MODEL - Model 1
modelvgg = VGG16(weights='imagenet')

# Load ResNet50 MODEL - Model 2
modelresnet = ResNet50(weights='imagenet')

# Load InceptionV3 MODEL - Model 3
modelinception = InceptionV3(weights='imagenet')

In [25]:
def classifyImage(model, imgPath, preprocessInputFunction, top_n=5, targetSize=(224, 224)):
    img = image.load_img(imgPath, target_size=targetSize)  #Resizes the image to the model specifications
    tempImg = image.img_to_array(img) 
    tempImg = np.expand_dims(tempImg, axis=0)
    tempImg = preprocessInputFunction(tempImg)
    predictionsRaw = model.predict(tempImg)  #Gets the top 5 predictions
    predictions = decode_predictions(predictionsRaw, top=top_n)[0]  #Decodes the predictions
    return predictions

In [26]:
folder_path = 'Task1Images' #Task 1 Pizzas Images path
files = os.listdir(folder_path)

resultsDictionary = {}

for fileName in files:
    resultsDictionary[fileName] = {}
    filePath = os.path.join(folder_path, fileName)
    vggPredictions = classifyImage(modelvgg, filePath, preprocess_input_vgg)
    resnetPredictions = classifyImage(modelresnet, filePath, preprocess_input_resnet)
    inceptionPredictions = classifyImage(modelinception, filePath, preprocess_input_inception, targetSize=(299, 299))
    resultsDictionary[fileName]['vgg'] = {}
    resultsDictionary[fileName]['resnet'] = {}
    resultsDictionary[fileName]['inception'] = {}

    for _, label, score in vggPredictions:
        resultsDictionary[fileName]['vgg'][label] = float(round(score,2))
    for _, label, score in resnetPredictions:
        resultsDictionary[fileName]['resnet'][label] = float(round(score,2))
    for _, label, score in inceptionPredictions:
        resultsDictionary[fileName]['inception'][label] = float(round(score,2))

1/1 [==============================] - 0s 72ms/step


In [27]:
print("\nResults Dictionary:")
print(resultsDictionary)

#Save dictionary to a JSON File
with open('Task1Results.json', 'w') as fp:
    json.dump(resultsDictionary, fp)


Results Dictionary:
{'AbnormalPizza.jpg': {'vgg': {'pizza': 0.5400000214576721, 'soup_bowl': 0.3100000023841858, 'consomme': 0.10999999940395355, 'potpie': 0.009999999776482582, 'plate': 0.0}, 'resnet': {'pizza': 0.8999999761581421, 'soup_bowl': 0.03999999910593033, 'plate': 0.029999999329447746, 'zucchini': 0.009999999776482582, 'consomme': 0.0}, 'inception': {'pizza': 0.7900000214576721, 'broccoli': 0.05999999865889549, 'bagel': 0.009999999776482582, 'plate': 0.0, 'mashed_potato': 0.0}}, 'FrenchBreadPizza.jpg': {'vgg': {'pizza': 0.5600000023841858, 'pomegranate': 0.07000000029802322, 'binder': 0.07000000029802322, 'strawberry': 0.05000000074505806, 'cleaver': 0.019999999552965164}, 'resnet': {'pizza': 0.8600000143051147, 'meat_loaf': 0.10999999940395355, 'pomegranate': 0.0, 'toaster': 0.0, 'refrigerator': 0.0}, 'inception': {'pizza': 0.8100000023841858, 'pomegranate': 0.019999999552965164, 'French_loaf': 0.0, 'bow_tie': 0.0, 'spatula': 0.0}}, 'FullPizzaHalfImg.jpg': {'vgg': {'bottle

## Data Plots

In [7]:
import matplotlib.pyplot as plt
import json
import os
import pandas as pd

In [8]:
with open('Task1Results-Final.json', 'r') as file:
    json_data = json.load(file)

output_folder = 'output_plots'
os.makedirs(output_folder, exist_ok=True)

In [12]:
rows = []

for image, models in json_data.items():
    for model, items in models.items():
        for item, score in items.items():
            # Append data as a dictionary to the list if the score is not 0.0
            if score != 0.0:
                rows.append({"Image": image, "Model": model, "Item": item, "Score": score})

df = pd.DataFrame(rows)

top_3_df = df.groupby(['Image', 'Model']).apply(lambda x: x.nlargest(3, 'Score')).reset_index(drop=True)

for image, image_df in top_3_df.groupby('Image'):
    img_name, _ = os.path.splitext(image)
    
    plt.figure(figsize=(10, 6))
    for model, model_df in image_df.groupby('Model'):
        plt.bar(model_df['Item'], model_df['Score'], label=model)

    plt.xlabel('Item')
    plt.ylabel('Score')
    plt.title(f'Top Predictions for {img_name}')
    plt.legend()
    plt.ylim(0, 1)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    plt.savefig(os.path.join(output_folder, f'{img_name}_comparison.png'))
    
    # plt.show()
    
    plt.close()

In [10]:
# Filter the DataFrame for pizza only
pizza_df = df[df['Item'] == 'pizza']

pizza_accuracy = pizza_df.groupby('Model')['Score'].mean().reset_index()

pizza_accuracy = pizza_accuracy.sort_values(by='Score', ascending=False)

plt.figure(figsize=(10, 6))
plt.bar(pizza_accuracy['Model'], pizza_accuracy['Score'], color='skyblue')
plt.xlabel('Model')
plt.ylabel('Mean Accuracy for Pizza')
plt.title('Accuracy of Identifying Pizza for Each Model')
plt.ylim(0, 1)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

plt.savefig(os.path.join(output_folder, 'overall_performance.png'))

# plt.show()

plt.close()

In [11]:
# Group by Model and Item, calculate the average score for each group
result_df = df.groupby(['Model', 'Item'])['Score'].mean().reset_index()

top_5_df = result_df.groupby('Model').apply(lambda x: x.nlargest(5, 'Score')).reset_index(drop=True)

table = top_5_df.pivot(index='Model', columns='Item', values='Score').fillna(0)

table = table[table.sum().sort_values(ascending=False).index]

plt.figure(figsize=(10, 6))
plt.imshow(table, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Average Score')
plt.xticks(range(len(table.columns)), table.columns, rotation=45, ha='right')
plt.yticks(range(len(table.index)), table.index)
plt.title('Top Predictions for Each Model')
plt.tight_layout()

plt.savefig(os.path.join(output_folder, 'top_predictions.png'))

# plt.show()

plt.close()